# Import

In [1]:
%matplotlib inline
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.exceptions import DataConversionWarning
# from tqdm import tqdm
# data Process
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
# model setup
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# metric
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, accuracy_score, recall_score, roc_auc_score, confusion_matrix
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=RuntimeWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
# df = pd.read_csv('/home/ec2-user/SageMaker/data/train_features.csv')
df = pd.read_csv('data/train_features.csv')
df2 = pd.read_csv('data/test_features.csv')
df3 = pd.read_csv('data/train_labels.csv')

## Useful functions

In [4]:
def submission(y_pred,X_test,outPath):
  id=pd.DataFrame(X_test.id,columns=['id']).astype('int32')
  prediction=pd.DataFrame(y_pred,columns=['charged_off'])
  final=pd.concat([id, prediction], axis=1).to_csv(outPath, index=False, 
                                                   header=['id','charged_off'])

# Preview Data

In [5]:
# preview data
print("df shape:"), print(df.shape), print("---"*20)
print("df2 shape:"), print(df2.shape), print("---"*20)
print("df3 shape:"), print(df3.shape), print("---"*20)

df shape:
(1309457, 103)
------------------------------------------------------------
df2 shape:
(26724, 103)
------------------------------------------------------------
df3 shape:
(1309457, 2)
------------------------------------------------------------


(None, None, None)

In [6]:
print("df columns:"), print(df.columns), print("---"*20)
print("df select_dtypes(include=[np.number]).columns.values:"), print(df.select_dtypes(include=[np.number]).columns.values), print("---"*20)
print("df select_dtypes(exclude=[np.number]).columns:"), print(df.select_dtypes(exclude=[np.number]).columns), print("---"*20)
print("df dtypes.sort_values(ascending=False):"), print(df.dtypes.sort_values(ascending=False)), print("---"*20)
print("df isnull().sum().sum():"), print(df.isnull().sum().sum()), print("---"*20)
print("df isna().sum().sort_values(ascending=False):"), print(df.isna().sum().sort_values(ascending=False)), print("---"*20)

df columns:
Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       ...
       'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'disbursement_method'], dtype='object', length=103)
------------------------------------------------------------
df select_dtypes(include=[np.number]).columns.values:
['id' 'member_id' 'loan_amnt' 'funded_amnt' 'installment' 'annual_inc'
 'url' 'dti' 'delinq_2yrs' 'inq_last_6mths' 'mths_since_last_delinq'
 'mths_since_last_record' 'open_acc' 'pub_rec' 'revol_bal' 'total_acc'
 'collections_12_mths_ex_med' 'mths_since_last_major_derog'
 'annual_inc_joint' 'dti_joint' 'acc_now_delinq' 'tot_coll_amt'
 'tot_cur_bal' 'open_acc_6m' 'open_act_il' 'open_il_12m' 'open_il_24m'
 'mths_since_rcnt_il' 't

(None, None, None)

In [2]:
# list with more than 660359 NaN
list1 = ["url", "member_id", "sec_app_mths_since_last_major_derog", "sec_app_revol_util", "revol_bal_joint", 
         "sec_app_earliest_cr_line", "sec_app_inq_last_6mths", "sec_app_mort_acc", "sec_app_open_acc", 
         "sec_app_open_act_il", "sec_app_num_rev_accts", "sec_app_chargeoff_within_12_mths", 
         "sec_app_collections_12_mths_ex_med", "dti_joint", "annual_inc_joint", "desc", "mths_since_last_record", 
         "mths_since_recent_bc_dlq", "mths_since_last_major_derog", "mths_since_recent_revol_delinq", "il_util", 
         "mths_since_rcnt_il", "all_util", "total_cu_tl", "inq_last_12m", "open_acc_6m", "open_il_12m", 
         "open_il_24m", "total_bal_il", "open_rv_12m", "open_rv_24m", "max_bal_bc", "open_act_il", "inq_fi", 
         "mths_since_last_delinq"]

# df = df.drop(columns=list1)
# df2 = df2.drop(columns=list1)

In [8]:
df.head().T

,0,1,2,3,4
id,1246716,1678744,1149018,1349740,1234365
loan_amnt,22000,10000,7000,1400,19200
funded_amnt,22000,10000,7000,1400,19200
term,36 months,36 months,36 months,36 months,36 months
int_rate,5.32%,5.32%,6.97%,13.67%,8.18%
installment,662.53,301.15,216.05,47.63,603.26
grade,A,A,A,C,B
sub_grade,A1,A1,A3,C4,B1
emp_title,Transition Coordinator,Director HQ Controlling,NaN,Administrative Assistant,Plant Electrical Engineer
emp_length,10+ years,4 years,NaN,5 years,< 1 year


In [9]:
df2.head().T

,0,1,2,3,4
id,2264704,1896343,2077525,1846827,1210251
loan_amnt,20000,4000,10000,12000,2400
funded_amnt,20000,4000,10000,12000,2400
term,36 months,36 months,36 months,60 months,36 months
int_rate,8.18%,13.99%,10.99%,14.31%,6.72%
installment,628.39,136.7,327.34,281.16,73.8
grade,B,C,B,C,A
sub_grade,B1,C1,B2,C4,A3
emp_title,Superintendent,Seats Inc,Registered Nurse,Personal Banker,Respiratory Therapist
emp_length,2 years,5 years,6 years,2 years,5 years


In [10]:
df3.head().T

,0,1,2,3,4
id,1246716,1678744,1149018,1349740,1234365
charged_off,0,0,1,0,0


In [11]:
df = df.select_dtypes(include=[np.number])
df.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc',
       'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit'],
      dtype='object')

In [12]:
for i in df.columns:
    df[i] = df[i].fillna(df[i].mean())
df.isna().sum().sort_values(ascending=False)

total_il_high_credit_limit    0
total_bc_limit                0
mo_sin_old_il_acct            0
delinq_amnt                   0
chargeoff_within_12_mths      0
bc_util                       0
bc_open_to_buy                0
avg_cur_bal                   0
acc_open_past_24mths          0
total_rev_hi_lim              0
tot_cur_bal                   0
tot_coll_amt                  0
acc_now_delinq                0
collections_12_mths_ex_med    0
total_acc                     0
revol_bal                     0
pub_rec                       0
open_acc                      0
inq_last_6mths                0
delinq_2yrs                   0
dti                           0
annual_inc                    0
installment                   0
funded_amnt                   0
loan_amnt                     0
mo_sin_old_rev_tl_op          0
mo_sin_rcnt_rev_tl_op         0
mo_sin_rcnt_tl                0
num_sats                      0
total_bal_ex_mort             0
tot_hi_cred_lim               0
tax_lien

# Baseline Model Prediction

In [16]:
################################ 
submission_url = 'Model1-prediction-baseline0.csv'

# 0 Data Source
train_features = pd.read_csv('data/train_features.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test_features = pd.read_csv('data/test_features.csv')
# sample_submission = pd.read_csv('sample_submission.csv')

# 1 Set X & y - train & test
y_train = train_labels['charged_off']
X_train = train_features
X_test  = test_features

# 2 Data Wrangle
X_train = X_train.drop(columns=list1)
X_test = X_test.drop(columns=list1)
X_train = X_train[list(X_train.select_dtypes(include=[np.number]))]
X_test = X_test[list(X_train.select_dtypes(include=[np.number]))]
for i in X_train.columns:
    X_train[i] = X_train[i].fillna(X_train[i].mean())
for i in X_test.columns:
    X_test[i] = X_test[i].fillna(X_test[i].mean())
print("X_train.shape:",X_train.shape)
print("X_test.shape:",X_test.shape)

# 3 Set X & y - train & validation from X & y train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# 4 Make pipeline
pipeline = make_pipeline(
    RobustScaler(),
    SelectKBest(f_classif),
    LogisticRegression(random_state=0, solver='lbfgs')
)

param_grid = {
#     'selectkbest__k': range(4, len(X_train.columns)+1),
    'selectkbest__k': range(4, len(X_train.columns)+1),    
    }

# 5 Fit on the train set
gs = GridSearchCV(pipeline, param_grid=param_grid, cv=2, 
                  scoring='roc_auc', 
                  iid=False,
                  verbose=2)

gs.fit(X_train, y_train)

# 6 Validation Results
validation_score = gs.best_score_
print('Validation Score: ', validation_score)
print('Best parameter:', gs.best_params_)
print('Best estimator:', gs.best_estimator_)

# 7 Print which features was selected
selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names=all_names[selected_mask]
unselected_names = all_names[~selected_mask]
print("-"*100)

print(all_names)
print("-"*100)
print('Features selected:')
for name in selected_names:
  print(name)
print("-"*100)

print("Features not selected:")
for name in unselected_names:
  print(name)
print("-"*100)

# 8 Make Prediction and Save submission
y_pred=gs.predict(X_test)
# for google colab
# submission_url = '/content/drive/My Drive/Colab Notebooks/Model1-prediction.csv'
submission(y_pred,test_features,submission_url)

print("train_labels.shape:",train_labels.shape)
print("submission csv shape:")
pd.read_csv(submission_url).shape

X_train.shape: (1309457, 52)
X_test.shape: (26724, 52)
(982092, 52) (327365, 52) (982092,) (327365,)
Fitting 2 folds for each of 49 candidates, totalling 98 fits
[CV] selectkbest__k=4 ................................................
[CV] ................................. selectkbest__k=4, total=   6.1s
[CV] selectkbest__k=4 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.7s remaining:    0.0s


[CV] ................................. selectkbest__k=4, total=   4.4s
[CV] selectkbest__k=5 ................................................
[CV] ................................. selectkbest__k=5, total=   4.2s
[CV] selectkbest__k=5 ................................................
[CV] ................................. selectkbest__k=5, total=   4.2s
[CV] selectkbest__k=6 ................................................
[CV] ................................. selectkbest__k=6, total=   4.4s
[CV] selectkbest__k=6 ................................................
[CV] ................................. selectkbest__k=6, total=   4.5s
[CV] selectkbest__k=7 ................................................
[CV] ................................. selectkbest__k=7, total=   4.5s
[CV] selectkbest__k=7 ................................................
[CV] ................................. selectkbest__k=7, total=   4.5s
[CV] selectkbest__k=8 ................................................
[CV] .

[CV] ................................ selectkbest__k=33, total=  14.3s
[CV] selectkbest__k=34 ...............................................
[CV] ................................ selectkbest__k=34, total=  14.5s
[CV] selectkbest__k=34 ...............................................
[CV] ................................ selectkbest__k=34, total=  14.9s
[CV] selectkbest__k=35 ...............................................
[CV] ................................ selectkbest__k=35, total=  15.8s
[CV] selectkbest__k=35 ...............................................
[CV] ................................ selectkbest__k=35, total=  15.1s
[CV] selectkbest__k=36 ...............................................
[CV] ................................ selectkbest__k=36, total=  14.2s
[CV] selectkbest__k=36 ...............................................
[CV] ................................ selectkbest__k=36, total=  14.1s
[CV] selectkbest__k=37 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  98 out of  98 | elapsed: 21.7min finished


Validation Score:  0.6788866019813439
Best parameter: {'selectkbest__k': 41}
Best estimator: Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k=41, score_func=<function f_classif at 0x7f1b3119a9d8>)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))])
----------------------------------------------------------------------------------------------------
Index(['id', 'loan_amnt', 'funded_amnt', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 

In [3]:
################################ 
submission_url = 'Model1-prediction-baseline1.csv'

# 0 Data Source
train_features = pd.read_csv('data/train_features.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test_features = pd.read_csv('data/test_features.csv')
# sample_submission = pd.read_csv('sample_submission.csv')

# 1 Set X & y - train & test
y_train = train_labels['charged_off']
X_train = train_features
X_test  = test_features

# 2 Data Wrangle
X_train = X_train.drop(columns=list1)
X_test = X_test.drop(columns=list1)
X_train = X_train[list(X_train.select_dtypes(include=[np.number]))]
X_test = X_test[list(X_train.select_dtypes(include=[np.number]))]
for i in X_train.columns:
    X_train[i] = X_train[i].fillna(X_train[i].mean())
for i in X_test.columns:
    X_test[i] = X_test[i].fillna(X_test[i].mean())
print("X_train.shape:",X_train.shape)
print("X_test.shape:",X_test.shape)

# 3 Set X & y - train & validation from X & y train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# 4 Make pipeline
pipeline = make_pipeline(
    RobustScaler(),
    SelectKBest(f_classif),
    RandomForestClassifier(n_estimators=500)
)

param_grid = {
#     'selectkbest__k': range(4, len(X_train.columns)+1),
    'selectkbest__k': [41],    
    }

# 5 Fit on the train set
gs = GridSearchCV(pipeline, param_grid=param_grid, cv=2, 
                  scoring='roc_auc', 
                  iid=False,
                  verbose=2)

gs.fit(X_train, y_train)

# 6 Validation Results
validation_score = gs.best_score_
print('Validation Score: ', validation_score)
print('Best parameter:', gs.best_params_)
print('Best estimator:', gs.best_estimator_)

# 7 Print which features was selected
selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names=all_names[selected_mask]
unselected_names = all_names[~selected_mask]
print("-"*100)

print(all_names)
print("-"*100)
print('Features selected:')
for name in selected_names:
  print(name)
print("-"*100)

print("Features not selected:")
for name in unselected_names:
  print(name)
print("-"*100)

# 8 Make Prediction and Save submission
y_pred=gs.predict(X_test)
# for google colab
# submission_url = '/content/drive/My Drive/Colab Notebooks/Model1-prediction.csv'
submission(y_pred,test_features,submission_url)

print("train_labels.shape:",train_labels.shape)
print("submission csv shape:")
pd.read_csv(submission_url).shape

X_train.shape: (1309457, 52)
X_test.shape: (26724, 52)
(982092, 52) (327365, 52) (982092,) (327365,)
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] selectkbest__k=41 ...............................................
[CV] ................................ selectkbest__k=41, total=25.7min
[CV] selectkbest__k=41 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 27.9min remaining:    0.0s


[CV] ................................ selectkbest__k=41, total=25.6min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 55.7min finished


Validation Score:  0.6893237469573033
Best parameter: {'selectkbest__k': 41}
Best estimator: Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k=41, score_func=<function f_classif at 0x7fe38503c7b8>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, cr...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
----------------------------------------------------------------------------------------------------
Index(['id', 'loan_amnt', 'funded_amnt', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_s

NameError: name 'submission' is not defined

In [5]:
submission(y_pred,test_features,submission_url)

print("train_labels.shape:",train_labels.shape)
print("submission csv shape:")
pd.read_csv(submission_url).shape

train_labels.shape: (1309457, 2)
submission csv shape:


(26724, 2)